In [2]:
!pip install pyarrow

# Q1

In [3]:
import pandas as pd

In [4]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [5]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [6]:
print("Number of Columns in January 2023 data:",df_jan.shape[1])

Number of Columns in January 2023 data: 19


# Q2

In [7]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds() / 60)

In [8]:
print("Standard deviation of the trips duration in January: {:.2f}".format(df_jan.duration.std()))

Standard deviation of the trips duration in January: 42.59


# Q3

In [9]:
df = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
print("Fraction of the records left after the outliers dropped:",(df.shape[0]/df_jan.shape[0])*100,'%')

Fraction of the records left after the outliers dropped: 98.1220282212598 %


# Q4

In [10]:
from sklearn.feature_extraction import DictVectorizer

categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)
dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X = dv.fit_transform(dicts)

/tmp/ipykernel_43012/1971019528.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [11]:
print("Dimensionality of the matrix(number of columns):",X.shape[1])

Dimensionality of the matrix(number of columns): 515


# Q5

In [12]:
y = df['duration'].values

In [13]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)

In [14]:
from sklearn.metrics import mean_squared_error
import numpy as np

rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"RMSE on training data: {rmse:.2f}")

RMSE on training data: 7.65


# Q6

In [15]:
validation_data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [16]:
validation_data['duration'] = (validation_data.tpep_dropoff_datetime - validation_data.tpep_pickup_datetime).dt.total_seconds() / 60
validation_data = validation_data[(validation_data.duration >= 1) & (validation_data.duration <= 60)]
validation_data[categorical] = validation_data[categorical].astype(str)

In [17]:
from scipy.sparse import vstack

def batch_transform(data, batch_size=500_000):
    matrices = []
    for i in range(0, len(data), batch_size):
        batch = data.iloc[i:i+batch_size]
        dicts = batch[categorical].to_dict(orient='records')
        X_batch = dv.transform(dicts)
        matrices.append(X_batch)
    return vstack(matrices)

X_val = batch_transform(validation_data)

In [18]:
y_val = validation_data['duration'].values
y_pred = model.predict(X_val)

In [19]:
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE on validation data: {rmse_val:.2f}")

RMSE on validation data: 7.81
